In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from splinter import Browser
import time

## NASA Mars News

In [2]:
html = requests.get("https://mars.nasa.gov/news/").text
soup = BeautifulSoup(html, 'lxml')

In [3]:
news_title = soup.find('div', class_='content_title').text.strip()
news_p = soup.find('div', class_='rollover_description_inner' ).text.strip()

In [4]:
news_title, news_p

("Mount Sharp 'Photobombs' Mars Curiosity Rover",
 "A new self-portrait of NASA's Curiosity Mars rover shows the vehicle on Vera Rubin Ridge.")

## JPL Mars Space Images - Featured Image

In [5]:
browser = Browser("chrome",headless="False")
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
#Button that opens featured image happens to be first instance of id full_image
browser.find_by_id('full_image').first.click()
#Adding in sleep probably not necessary
time.sleep(2)
#a button with more info on it takes you to the page for the image
browser.find_link_by_partial_text('more info').first.click()
time.sleep(1)
#click on the image and save the URL
browser.find_by_css('img.main_image').first.click()
image_url = browser.url
print(image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18847_hires.jpg


## Mars Weather

In [6]:
html = requests.get("https://twitter.com/marswxreport?lang=en").text
soup = BeautifulSoup(html, 'lxml')
tweet_list = soup('p', 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
#Not all tweets are weather reports; all weather reports seem to start with 'Sol'
mars_weather = ''
weather_found = False
weather_index = 0
while weather_found == False:
    if "Sol" in tweet_list[weather_index].text.strip():
        mars_weather = tweet_list[weather_index].text.strip()
        weather_found = True
    else:
        weather_index += 1

In [7]:
mars_weather

'Sol 1951 (Jan 31, 2018), Sunny, high -23C/-9F, low -77C/-106F, pressure at 7.49 hPa, daylight 05:42-17:28'

## Mars Facts

In [8]:
html = requests.get("https://space-facts.com/mars/").text
#soup = BeautifulSoup(html, 'lxml')
table = pd.read_html(html)[0]
table_series = pd.Series(list(table[1]), index = list(table[0]) )
table = table.rename(columns =  { 0:"Feature", 1:"Value" })
table = table.set_index('Feature')

In [9]:
table_html = table.to_html()
print(table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Feature</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <

## Mars Hemispheres

In [21]:
browser = Browser("chrome", headless="False")
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
links = browser.find_link_by_partial_text('Enhanced')
url_list = []
title_url_list = []
for j in range(4):
    browser.find_link_by_partial_text('Enhanced')[j].click()
    time.sleep(1)
    browser.click_link_by_partial_href('jpg')
    time.sleep(1)
    url_list.append(browser.windows[1].url)
    title_url_list.append(browser.windows[0].url)
    browser.windows[1].close()
    browser.click_link_by_partial_text("Back")

hemisphere_list = [title_url_list[j].split('/')[7].split('_enhanced')[0] for j in range(4)]
    

In [22]:
hemisphere_dicts = [ { "title" : hemisphere_list[j] , "img_url": url_list[j]  } for j in range(4) ]

In [23]:
hemisphere_dicts

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'cerberus'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'schiaparelli'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'syrtis_major'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'valles_marineris'}]

In [16]:
table

,Value
Feature,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers
